In [2]:
import numpy as np

In [27]:
model_folder = "../training/saved/2024-03-28/5_1_8_octaves"
archive = np.load(model_folder + "/predictions.npz")
y_pred = archive["y_pred"]
y_gt = archive["y_gt"]

In [28]:
def tab_to_pitch(tab):
    result = []
    open_pitch_values = [40, 45, 50, 55, 59, 64]
    for index, string_vector in enumerate(tab):
        fret_index = np.argmax(string_vector)
        #fret_index = 0 means that the string is muted
        if fret_index > 0:
            midi_value = fret_index + open_pitch_values[index] - 1
            if midi_value not in result:
                result.append(midi_value)
    return result

In [29]:
# aligns equal pitches
def process_pitches(pred_pitch_list, gt_pitch_list):
    i = 0
    j = 0
    while i < len(pred_pitch_list) and j < len(gt_pitch_list):
        if pred_pitch_list[i] > gt_pitch_list[j]:
            pred_pitch_list = pred_pitch_list[:i] + [0] + pred_pitch_list[i:]
        elif pred_pitch_list[i] < gt_pitch_list[j]:
            gt_pitch_list = gt_pitch_list[:j] + [0] + gt_pitch_list[j:]

        i += 1
        j += 1
    ppllen = len(pred_pitch_list)
    gpllen = len(gt_pitch_list)
    pred_pitch_list += [0] * (gpllen-j)
    gt_pitch_list += [0] * (ppllen-i)
    return pred_pitch_list, gt_pitch_list

In [30]:
pred = []
gt = []

for i in range(len(y_gt)):
    pred_pitches = tab_to_pitch(y_pred[i])
    gt_pitches = tab_to_pitch(y_gt[i])
    pred_pitches.sort()
    gt_pitches.sort()

    pred_pitches, gt_pitches = process_pitches(pred_pitches, gt_pitches)
    if (len(pred_pitches) != len(gt_pitches)):
        raise "Hello?"

    pred += pred_pitches
    gt += gt_pitches

In [31]:
gt_freq = [0] * 100
pred_freq = [0] * 100

for i in range(len(gt)):
    gti = gt[i]
    predi = pred[i]
    gt_freq[gti] += 1
    if gti == predi:
        pred_freq[predi] += 1
avg = 0
k = 0
for i in range(40, 83):
    if gt_freq[i] > 0:
        avg += pred_freq[i]/gt_freq[i]
        k += 1
        print(f"pitch {i}: {pred_freq[i]/gt_freq[i]}")
print(avg/k)

pitch 40: 0.9155264723378941
pitch 41: 0.9564575645756458
pitch 42: 0.8580508474576272
pitch 43: 0.9568913175470553
pitch 44: 0.9136069114470843
pitch 45: 0.7670273711012094
pitch 46: 0.9190225203641591
pitch 47: 0.9165070854078897
pitch 48: 0.967741935483871
pitch 49: 0.9515418502202643
pitch 50: 0.8275862068965517
pitch 51: 0.9060773480662984
pitch 52: 0.8493975903614458
pitch 53: 0.9309426229508196
pitch 54: 0.878584729981378
pitch 55: 0.6855016538037486
pitch 56: 0.8922776148582601
pitch 57: 0.8288088642659279
pitch 58: 0.8326334733053389
pitch 59: 0.6044254658385093
pitch 60: 0.8571428571428571
pitch 61: 0.8120010301313417
pitch 62: 0.7815495207667732
pitch 63: 0.7907761529808774
pitch 64: 0.7823115161557581
pitch 65: 0.7339296543359611
pitch 66: 0.707475317348378
pitch 67: 0.759751552795031
pitch 68: 0.5582341891045711
pitch 69: 0.7191283292978208
pitch 70: 0.6966887417218544
pitch 71: 0.7764945652173914
pitch 72: 0.6331853496115427
pitch 73: 0.7985524728588661
pitch 74: 0.682363